This program executes one simulation using initial parameters that were set in 'main_code.ipynb' and are saved and imported from 'initial_parameters.py'.

# Import libraries

In [1]:
#import hoomd that ist saved on computer
import sys
sys.path.append("/net/theorie/home/niklas.butkevich/miniconda2/envs/py3/lib/python3.7/site-packages")
import hoomd
import hoomd.md

import numpy as np
import math
import os

# Initialize the system

In [2]:
hoomd.context.initialize("")

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU


# Define the parameters (that were set in 'main_code.ipynb')

In [3]:
#define number of polymers segments (monomers)
Monomers = 100
#define number of bonds between segments
Bonds = Monomers-1

#define spring constant (Rouse model) for bonds between segments in units energy/distance^2
k = 1

#define number of passive particles
n_passive_particles = 0

#define number of active particles
n_active_particles = 0

#define drag-coefficient constant gamma
gamma = 1

#define rest distance between segments
r0 = 0

#define temperature in units of kT
kT = 1

#define integration step time
dt = 1e-4

#calculate Kuhn length b
Kuhn_length_b = math.sqrt(3*kT/k)

#calculate Rouse time (Rouse mode = 1)
tau_rouse_theory = gamma*Monomers**2*Kuhn_length_b**2 / (3*np.pi**2*kT)

#define number of integration steps (e3 --> *1000, is written into files in correct digit-format)
integration_steps = int(tau_rouse_theory/dt*10)

#define number of points for graphical representation (number of callbacks)
number_of_points = 1000

#define period for analysing of quantities (write them in data file)
return_period = integration_steps/number_of_points

#define number of simulations (executions of this code)
number_of_simulations = 1

In [4]:
print (Kuhn_length_b)
print (tau_rouse_theory)
print (integration_steps)

1.7320508075688772
1013.2118364233777
101321183


In [5]:
#set epsilon and sigma for Lennard_jones potential between different particles
lj_epsilon_AA = 1
lj_sigma_AA = 1

#set cut-off radius for potential between two particles
r_cut = (2**(1/6))*lj_sigma_AA

# Create box with particles

Attention: Box has to be large, otherwise - if polmer is outside the box it appears on the opposite side - the end-to-end-values and other position parameters are calculated in a wrong way.

In [6]:
#define box dimensions
Lx = 3 * Monomers
Ly = 3 * Monomers
Lz = 3 * Monomers

#define the system as snapshot
snapshot = hoomd.data.make_snapshot(N = Monomers + n_passive_particles,
                                    
                                    #box-dimension: x-direction: -Lx/2 to Lx/2 etc.
                                    box=hoomd.data.boxdim(Lx=Lx, Ly=Ly, Lz=Lz),
                                    
                                    #list of types; typeid[0]='A', typeid[1]='B', etc.
                                    particle_types=['A'],
                                    bond_types=['polymer'])

# Set ID's positions and bonds for monomers/particles

In [7]:
#create list to set monomer's/particle's initial type-IDs, positions, bonds
initial_IDs = []
initial_positions = []
initial_bonds= []

#define initial quantities
initial_position_of_cm_x = 0
initial_position_of_cm_y = 0
initial_position_of_cm_z = 0

#set initial monomer positions and IDs
for monomer in range(Monomers):
    initial_IDs.append(0)
    initial_positions.append([monomer*1.5 - (Monomers/2) + 0.5, 0, 0])
    
    #calculate initial distance of center of mass from origin
    initial_position_of_cm_x += (initial_positions[monomer][0] / Monomers)
    initial_position_of_cm_y += (initial_positions[monomer][1] / Monomers)
    initial_position_of_cm_z += (initial_positions[monomer][2] / Monomers)
    
initial_position_of_cm = [initial_position_of_cm_x,
                          initial_position_of_cm_y,
                          initial_position_of_cm_z]

initial_end_to_end_vector = [initial_positions[Monomers-1][0]-initial_positions[0][0],
                             initial_positions[Monomers-1][1]-initial_positions[0][1],
                             initial_positions[Monomers-1][2]-initial_positions[0][2]]
    
for bond in range(Monomers-1):
    initial_bonds.append([bond, bond+1])
    
#resize the bonds list to actual number of bonds
snapshot.bonds.resize(Monomers-1)

#set initial passive particle positions and IDs
for passive_particle in range(n_passive_particles):
    initial_IDs.append(1)
    initial_positions.append([passive_particle - (n_passive_particles/2) + 0.5, 1, 0])
    
#copy IDs, positions and bonds into valid arrays
snapshot.particles.typeid[:] = initial_IDs
snapshot.particles.position[:] = initial_positions
snapshot.bonds.group[:] = initial_bonds

# Initialization of integration parameters

In [8]:
#initialize HOOMD using snapshot
system = hoomd.init.read_snapshot(snapshot)

#set the bond type and strength between monomers
harmonic = hoomd.md.bond.harmonic()
harmonic.bond_coeff.set('polymer', k=k, r0=r0)

#consider every particle
all = hoomd.group.all()

#safe the trajectory in 'results/datetime' to
#display in a visualisation program

hoomd.dump.gsd("trajectory.gsd",
               
               #return_period = number of points in plots
               period=return_period,
               group=all, overwrite=True)

notice(2): Group "all" created containing 100 particles


# Set Lennard-Jones potential

In [9]:
#cell method is used if r_cut is for all pairs of particles the same
nl = hoomd.md.nlist.cell()

#LJ-potential depens on r_cut
lj = hoomd.md.pair.lj(r_cut=r_cut, nlist=nl)

lj.pair_coeff.set('A', 'A', epsilon=lj_epsilon_AA, sigma=lj_sigma_AA)

# Set integrator and  parameters

In [10]:
#integrator
hoomd.md.integrate.mode_standard(dt=dt)

#dcsale is used to choose gamma
integrator = hoomd.md.integrate.brownian(group=all, kT=kT, dscale=False,
                                         seed=np.random.randint(0,9999))


#set gamma (friction constant); ATTENTION: set after integrator is set!
integrator.set_gamma('A', gamma=gamma)


notice(2): integrate.langevin/bd is using specified gamma values


# Run the simulation

In [ ]:
hoomd.run(integration_steps)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 1 exclusions             : 2
notice(2): Particles with 2 exclusions             : 98
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:10 | Step 45408 / 101321183 | TPS 4540.79 | ETA 06:11:43
Time 00:00:20 | Step 126851 / 101321183 | TPS 8143.75 | ETA 03:27:06
Time 00:00:30 | Step 226949 / 101321183 | TPS 10009.8 | ETA 02:48:19
Time 00:00:40 | Step 328439 / 101321183 | TPS 10148.1 | ETA 02:45:51
Time 00:00:50 | Step 430269 / 101321183 | TPS 10182.5 | ETA 02:45:08
Time 00:01:00 | Step 511313 / 101321183 | TPS 8103.63 | ETA 03:27:20
Time 00:01:10 | Step 606313 / 101321183 | TPS 9499.64 | ETA 02:56:41
Time 00:01:20 | Step 704288 / 101321183 | TPS 9796.41 | ETA 02:51:10
Time 00:01:30 | Step 801159 / 101321183 | TPS 9686.12 | ETA 02:52:57
Time 00:01:40 | Step 895874 / 101321183 | TPS 9471.24 | ETA 02:56:43
Time 00:01:50 | 